In [1]:
!ls

archive.zip  indian_food.csv  sample_data


In [2]:
from google.colab import files
files.upload()

Saving archive.zip to archive.zip


In [2]:
!unzip archive.zip

Archive:  archive.zip
replace indian_food.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n


In [3]:
!ls

archive.zip  indian_food.csv  sample_data


In [21]:
import csv
import random
import time

In [22]:
import torch
from torch import nn, optim, tensor
from torch.distributions.geometric import Geometric
import numpy as np

In [17]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [14]:
dictionaries = [ {} for _ in range(6)]

def ix2_onehot(ix, max):
  z = torch.zeros(max) #si gpu, aqui
  z[ix] = 1
  return z

def add2_dict(dictionary, key):
  key = key.lower()
  if key in dictionary:
    return 
  dictionary[key] = len(dictionary)

def add2_food_dict(ingridients):
  for ingridient in ingridients.split(','):
    ingridient =  ingridient.strip(" ")
    add2_dict(dictionaries[0], ingridient)

def read_data(path):
  X = []
  Y = []
  with open(path) as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    print(csv_reader.__next__())
    #['name', 'ingredients', 'diet', 'prep_time', 'cook_time', 'flavor_profile', 'course', 'state', 'region']
    max_prep_time = 0
    max_cook_time = 0
    for row in csv_reader:
      if not '-1' in row:
        add2_food_dict(row[1])
        add2_dict(dictionaries[1], row[2])
        max_prep_time = max(max_prep_time, float(row[3]))
        max_cook_time = max(max_cook_time, float(row[4]))
        add2_dict(dictionaries[2], row[5])
        add2_dict(dictionaries[3], row[6])
        add2_dict(dictionaries[4], row[7])
        add2_dict(dictionaries[5], row[8])
  with open(path) as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    csv_reader.__next__()
    for row in csv_reader:
      if not '-1' in row:
        p0 =  torch.zeros(len(dictionaries[0]))
        for ingridient in row[1].split(','):
          key =  ingridient.lower().strip(" ")
          p0[dictionaries[0][key]] = 1
        p1 = ix2_onehot(dictionaries[1][row[2].lower()], len(dictionaries[1]))
        p2 = tensor([float(row[3])/max_prep_time])
        p3 = tensor([float(row[4])/max_cook_time])
        p4 = ix2_onehot(dictionaries[2][row[5].lower()], len(dictionaries[2]))
        p5 = ix2_onehot(dictionaries[3][row[6].lower()], len(dictionaries[3]))
        s0 = ix2_onehot(dictionaries[4][row[7].lower()], len(dictionaries[4]))
        s1 = ix2_onehot(dictionaries[5][row[8].lower()], len(dictionaries[5]))

        X.append(torch.cat((p0, p1, p2, p3, p4, p5)))
        Y.append(torch.cat((s0, s1)))
    
    ten = list(zip(X, Y))
    random.shuffle(ten)
    X, Y = zip(*ten)

    corte = int(np.floor(len(X)*0.7))
    X, X_test = X[:corte], X[corte:]
    Y, Y_test = Y[:corte], Y[corte:]

    return (torch.stack(X),
            torch.stack(X_test),
            torch.stack(Y),
            torch.stack(Y_test))

In [15]:
X, X_test, Y, Y_test = read_data('indian_food.csv')

['name', 'ingredients', 'diet', 'prep_time', 'cook_time', 'flavor_profile', 'course', 'state', 'region']


In [48]:
def confusion(model, X, Y):
    '''
    Define una función de confusión con la que puedas comparar el resultado entre la red 
    que se entreno usando backpropagation y la red con mejor fitness resultante del algoritmo genético.
    '''
    salida = model.forward(X)
    tp = 0
    fp = 0
    fn = 0
    tn = 0
    H =  torch.round(salida)
    for (y, h) in zip(Y, H):
        for (yi, hi) in zip (y, h):
            if yi==hi:
                if hi==1:
                    tp = tp + 1
                else:
                    tn = tn + 1
            else:
                if hi==1:
                    fp = fp + 1
                else:
                    fn = fn + 1
    print("tp: ", tp)
    print("tn: ", tn)
    print("fp: ", fp)
    print("fn: ", fn)
    print("p:{:.2%} n:{:.2%} , t:{:.2%}".format(tp/(tp+fp+1),tn/(tn+fn+1),(tp+tn)/(tp+fp+tn+fn)))

In [55]:
class Gen_net(nn.Module):
    '''
    Definimos un modelo de red neuronal sobre el cual podamos 
    evaluar el algoritmo genético.
    '''
    def __init__(self, in_features, hidden_1, hidden_2, out_features):
        '''
        Define la estructura de la red como se define en el articulo.
        Ten en cuenta que, dado que no usamos el mismo conjunto de datos
        los valores de in_features y out_features se verán afectados.
        Agregar como parámetros estos valores te puede ayudar a reconstruir
        la red a partir de un cromosoma.
        '''
        super(Gen_net, self).__init__()
        self.to(device)
        self.layers=[in_features, hidden_1, hidden_2, out_features]
        self.fc1 = nn.Linear(in_features = in_features, out_features = hidden_1, bias=True)     
        self.fc2 = nn.Linear(in_features = hidden_1, out_features = hidden_2, bias=True) 
        self.fc3 = nn.Linear(in_features = hidden_2, out_features = out_features, bias=True)

    def forward(self, X):
        '''
        Define la función de feed forward, a partir de los datos
        de X. Usa una sigmoide entre las capas.
        '''
        X = self.fc1(X)
        X = nn.LeakyReLU(0.1)(X)
        X = nn.LeakyReLU(0.1)(self.fc2(X))
        X = nn.Sigmoid()(self.fc3(X))
        return X
    
    def build_from_chromosome(self, chromosome: torch.tensor):
        '''
        Define una función para que a partir de un cromosoma modifiques los 
        pesos de la red.        
        Se reciben los ch con todos los pesos de entrada a una neurona juntos
        ''' 
        p =  nn.Parameter
        i = 0
        
        j = torch.numel(self.fc1.weight)+torch.numel(self.fc1.bias)
        wb = torch.narrow(chromosome, 0, i, j).view(self.fc1.weight.shape[0],-1)
        self.fc1.weight = p(torch.narrow(wb, 1, 0, self.fc1.weight.shape[1]))
        self.fc1.bias = p(torch.flatten(torch.narrow(wb, 1, self.fc1.weight.shape[1], 1)))
        i = i + j
        
        
        j = torch.numel(self.fc2.weight)+torch.numel(self.fc2.bias)
        wb = torch.narrow(chromosome, 0, i, j).view(self.fc2.weight.shape[0],-1)
        self.fc2.weight = p(torch.narrow(wb, 1, 0, self.fc2.weight.shape[1]))
        self.fc2.bias = p(torch.flatten(torch.narrow(wb, 1, self.fc2.weight.shape[1], 1)))
        i = i + j
        
        j = torch.numel(self.fc3.weight)+torch.numel(self.fc3.bias)
        wb = torch.narrow(chromosome, 0, i, j).view(self.fc3.weight.shape[0],-1)
        self.fc3.weight = p(torch.narrow(wb, 1, 0, self.fc3.weight.shape[1]))
        self.fc3.bias = p(torch.flatten(torch.narrow(wb, 1, self.fc3.weight.shape[1], 1)))
        i = i + j        


In [73]:
class Gen_train():
    '''
    Define la estructura del algortimo genético
    '''
    def __init__(self, model, parent_scalar=0.5, operator_probabilities=0.1, population_size=50):
        '''
        Define las propiedades del algortimo como se describe en el artículo.
        Usa el parametro model para tener un ejemplar de la red sobre la cual puedas
        realizar las operaciones necesarias.
        Incializa la población utilizando una distribución t
        '''
        self.model = model 
        self.parent_scalar = parent_scalar
        self.operator_probabilities = operator_probabilities
        self.population_size = population_size
        torch.manual_seed(0)
        self.t =  torch.distributions.studentT.StudentT(torch.tensor([0.4]))
        self.population = list()
        chromosome = self.get_chromosome(model)
        for i in range(population_size):
            r = Gen_net(*model.layers)
            r.build_from_chromosome(self.t.sample((len(chromosome), )).reshape(-1))
            r.to(device)
            self.population.append(r)
        
    def get_weights(self, element):
        '''
        Función para obtener las matrices de pesos a partir de un elemento de la población.
        Debe tener las mismas dimenciónes que los pesos para la red.
        '''
        return (element.fc1.weight, element.fc2.weight, element.fc3.weight)
    
    def get_chromosome(self, element):
        flat = torch.flatten
        wb1 = torch.cat((element.fc1.weight.T, element.fc1.bias.view(1, -1))).T.cuda()
        wb2 = torch.cat((element.fc2.weight.T, element.fc2.bias.view(1, -1))).T.cuda()
        wb3 = torch.cat((element.fc3.weight.T, element.fc3.bias.view(1, -1))).T.cuda()
       
        chromosome = torch.cat((flat(wb1),flat(wb2),flat(wb3))).cuda()
        return chromosome
        

    def set_chromosome(self, w1, w2, w3, element):
        '''
        Función para actualizar un elemento de la población a partir de los pesos que determinan 
        a una red. Se actualizará el ejemplar de la población que se encuentre en la posición
        element.
        '''
        flat = torch.flatten
        chromosome = torch.cat(flat(w1), flat(w2), flat(w3)).cuda()
        element.build_from_chromosome(chromosome)
        
    
    def fitness(self, X, Y, element):
        '''
        Función para determinar la adecuación de un elemento de la población. 
        En esté caso, la adecuación nos servirá para determinar los elementos
        más aptos. Para está implementación se considerará que una adecuación 
        menor será de un menor individuo, por lo que tendrá mayores 
        probabilidades de reproducirse.        
        '''
        salida = element.feed_forward(X)
        error = nn.MSELoss()(salida,Y)
        return error
    
    ## Define las operaciónes que se usan en el experimento número 5 del artículo.
    ## Todas las definiciones de los operadores se encuentrán en la sección 5 del
    ## artículo.
    
    def crossover_nodes(self, r1, r2):
        c1 = self.get_chromosome(r1)
        c2 = self.get_chromosome(r2)
        layers = r1.layers
        
        hijo =  torch.tensor([]).cuda()
        beta=0
        for i, j in zip(layers[:-1], layers[1:]):
            i = i +1
            for alfa in range(j):
                if random.random()>0.5:
                    hijo = torch.cat((hijo, torch.narrow(c1, 0, alfa+beta, i)))
                else:
                    hijo = torch.cat((hijo, torch.narrow(c2, 0, alfa+beta, i)))
            beta = beta + j*i
        r = Gen_net(*model.layers)
        r.build_from_chromosome(hijo)
        r.to(device)
        return r
    
    def mutate_node(self, element):
        c = self.get_chromosome(element)
        layers = element.layers
        
        hijo =  torch.tensor([]).cuda()
        beta=0
        for i, j in zip(layers[:-1], layers[1:]):
            i = i + 1
            for alfa in range(j):
                neuron = torch.narrow(c, 0, alfa+beta, i)
                if random.random()<self.operator_probabilities:
                    for gamma in range(len(neuron)):
                        delta = self.t.sample().item()
                        neuron[gamma] = neuron[gamma] + delta
                hijo = torch.cat((hijo, neuron))    
            beta = beta + j*i
        r = Gen_net(*model.layers)
        
        r.build_from_chromosome(hijo)
        r.to(device)
        return r


    def get_ix(self, d):
      mutter = len(self.population)
      vater = len(self.population)
      while not mutter < len(self.population): 
        mutter = d.sample((1,)).view(-1).int().item()
      while (not vater < len(self.population)) or mutter == vater: 
        vater = d.sample((1,)).view(-1).int().item()
      return (mutter, vater)
      
        
    def train(self, steps, X, Y, batch=100):
        '''
        Define la función de entrenamiento, la cuál se realizará el número de pasos 
        seleccionados. Usa la variable de parent_scalar para determinar la probabilidad 
        de que un individuo de la población sea remplazado en cada iteración. Muestra 
        una gráfica del error al finalizar el entrenamiento. Y regresa una red con
        los pesos del individuo con mejor fitness al finalizar el entrenamiento.
        '''    
        def error(x):
            salida = x.forward(X)
            error =  nn.MSELoss()(salida,Y)
            return error.item()
        
        g = Geometric(torch.tensor([0.3]))
        t = time.time()
        for i in range(steps):
            #generamos una generaci\'on nueva
            genertation =  list()
            parents = self.population
            parents.sort(reverse = False, key = error) 

           
            mutter, vater = self.get_ix(g)
            
            sohn = self.crossover_nodes(self.population[mutter], self.population[vater])
            sohn = self.mutate_node(sohn)
            
            parents.pop()
            self.population = parents
            self.population.append(sohn)
        
            if(i%batch==batch-1):
                e = time.time()
                print("........... step: {} .... {} sec por step......".format(i, np.ceil(e-t)))
                print("error: {}".format(error(self.population[mutter])))
                i = np.random.randint(len(X)-3)
                _X = torch.narrow(X,0,i,3)
                _Y = torch.narrow(Y,0,i,3)
                confusion(self.population[mutter], _X, _Y)
                t = time.time()    

In [70]:
input_layer =  len(X[0])

In [71]:
output_layer = len(Y[0])

In [79]:
model = Gen_net(input_layer, 200, 200, output_layer)
model.to(device)
pop = Gen_train(model, operator_probabilities=0.3)
steps = 1000
pop.train(steps, X.cuda(), Y.cuda(), int(np.ceil(steps/50)))
confusion(pop.population[0], X_test.cuda(), Y_test.cuda())

........... step: 19 .... 62.0 sec por step......
error: 0.23968254029750824
tp:  0
tn:  74
fp:  10
fn:  6
p:0.00% n:91.36% , t:82.22%
........... step: 39 .... 63.0 sec por step......
error: 0.4126984179019928
tp:  3
tn:  50
fp:  34
fn:  3
p:7.89% n:92.59% , t:58.89%
........... step: 59 .... 63.0 sec por step......
error: 0.4126984179019928
tp:  2
tn:  50
fp:  34
fn:  4
p:5.41% n:90.91% , t:57.78%
........... step: 79 .... 62.0 sec por step......
error: 0.3113756775856018
tp:  1
tn:  61
fp:  23
fn:  5
p:4.00% n:91.04% , t:68.89%
........... step: 99 .... 64.0 sec por step......
error: 0.3759259283542633
tp:  1
tn:  48
fp:  36
fn:  5
p:2.63% n:88.89% , t:54.44%
........... step: 119 .... 64.0 sec por step......
error: 0.23968254029750824
tp:  1
tn:  76
fp:  8
fn:  5
p:10.00% n:92.68% , t:85.56%
........... step: 139 .... 63.0 sec por step......
error: 0.37433862686157227
tp:  3
tn:  53
fp:  31
fn:  3
p:8.57% n:92.98% , t:62.22%
........... step: 159 .... 63.0 sec por step......
error:

In [57]:
pop.train(steps, X.cuda(), Y.cuda(), int(np.ceil(steps/25)))
print("..............................................")
confusion(pop.population[0], X_test.cuda(), Y_test.cuda())

........... step: 19 .... 62.0 sec por step......
error: 0.2904762029647827
tp:  1
tn:  67
fp:  17
fn:  5
p:5.26% n:91.78% , t:75.56%
........... step: 39 .... 64.0 sec por step......
error: 0.20476190745830536
tp:  0
tn:  73
fp:  11
fn:  6
p:0.00% n:91.25% , t:81.11%
........... step: 59 .... 61.0 sec por step......
error: 0.21798941493034363
tp:  3
tn:  72
fp:  12
fn:  3
p:18.75% n:94.74% , t:83.33%
........... step: 79 .... 63.0 sec por step......
error: 0.2904762029647827
tp:  2
tn:  58
fp:  26
fn:  4
p:6.90% n:92.06% , t:66.67%
........... step: 99 .... 64.0 sec por step......
error: 0.20476190745830536
tp:  1
tn:  68
fp:  16
fn:  5
p:5.56% n:91.89% , t:76.67%
........... step: 119 .... 62.0 sec por step......
error: 0.20476190745830536
tp:  3
tn:  73
fp:  11
fn:  3
p:20.00% n:94.81% , t:84.44%
........... step: 139 .... 61.0 sec por step......
error: 0.20714285969734192
tp:  3
tn:  66
fp:  18
fn:  3
p:13.64% n:94.29% , t:76.67%


KeyboardInterrupt: ignored

In [44]:
torch.cuda.empty_cache() 